<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Evaluate machine learning models</h1>
<h3 align="right"><a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso/blob/main/notebooks/analyse/Evaluate_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h3>
<h3 align="right">Written by the KSO Team</h3>

# Set up KSO requirements

### Install all the requirements

Installing the requirements in Google Colab takes ~4 mins and might automatically crash/restart the session. Please run this cell until you get the "KSO successfully imported!" message.

In [ ]:
%matplotlib inline

def initiate_dev_version():
    kso_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
    if os.path.isdir(os.path.join(kso_path, "kso_utils")):
        sys.path.insert(0, kso_path)
        %load_ext autoreload
        %autoreload 2
        print("Development mode ON - kso-utils added to the system.")
    else:
        raise FileNotFoundError("kso_utils directory not found in the expected path.")

def install_kso_utils():
    !pip install -q kso-utils
    # Temporary workaround to install panoptes from the source (avoid requests incompatibility)
    !pip install git+https://github.com/zooniverse/panoptes-python-client.git
    print("Restarting runtime to apply package changes...")
    os.kill(os.getpid(), 9)

try:
    import kso_utils.widgets as kso_widgets
    import kso_utils.project_utils as p_utils
    import kso_utils.yolo_utils as y_utils
    from kso_utils.project import ProjectProcessor, MLProjectProcessor
    print("KSO successfully imported!")
except Exception as e:
    print(f"Error importing kso modules: {e}")
    try:
        initiate_dev_version()
        import kso_utils.widgets as kso_widgets
        import kso_utils.project_utils as p_utils
        import kso_utils.yolo_utils as y_utils
        from kso_utils.project import ProjectProcessor, MLProjectProcessor
        print("KSO successfully imported!")
    except Exception as e:
        install_kso_utils()

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

### Initiate project's database

In [ ]:
# Find project
project = p_utils.find_project(project_name=project_name.value)
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

# Evaluate model on custom footage

### Choose the model

In [ ]:
model = mlp.choose_model()

### Choose folder to download the model

In [ ]:
download_dir = kso_widgets.choose_folder(".", "where to download the model")

### Download model

In [ ]:
artifact_dir = mlp.get_model(model.value, download_dir.selected)

### Choose the footage to run the models into

In [ ]:
pp.choose_footage_source()

In [ ]:
pp.choose_footage()

### Choose a confidence threshold for evaluation

In [ ]:
conf_thres = kso_widgets.choose_conf()

## Choose a suitable experiment name

In [ ]:
exp_name = kso_widgets.choose_experiment_name()

### Choose a suitable folder to store model output

In [ ]:
save_dir = kso_widgets.choose_folder(".", "where to store model output")

### Run model over selected custom footage

In [ ]:
# Ensure the selected footage and paths are loaded to the system
pp.check_selected_movies()

# Get the paths of the movies selected
mlp.detect_yolo(
    save_dir=save_dir.selected,
    conf_thres=conf_thres.value,
    artifact_dir=artifact_dir,
    save_output=True,
    project=mlp.project_name,
    name=exp_name.value,
    model=model.value,
    latest=False,
    source=(
        pp.selected_movies_paths
        if isinstance(pp.selected_movies_paths, str)
        else pp.selected_movies_paths[0]
    ),
)

### View model output

In [ ]:
viewer = y_utils.choose_files(mlp.eval_dir)

### Investigate training and validation datasets (only image data)

In [ ]:
train_dataset, val_dataset = mlp.get_dataset(model.value)

In [ ]:
# Training set
y_utils.get_data_viewer(os.path.join(train_dataset, "data/images"))

In [ ]:
# Validation set
y_utils.get_data_viewer(os.path.join(val_dataset, "data/images"))

# Track unique individuals (Optional)

In [ ]:
mlp.track_individuals(
    name=mlp.project_name,
    source=pp.selected_movies_paths,
    artifact_dir=artifact_dir,
    conf_thres=conf_thres.value,
    img_size=(640, 640),
)

# (Optional) Clean noisy tracking output

In [ ]:
# Choose another evaluation directory
mlp.eval_dir = kso_widgets.choose_folder(".", "tracking folder")

In [ ]:
y_utils.adjust_tracking(
    tracking_folder=mlp.eval_dir.selected,
    avg_diff_frames=90,
    min_frames_length=90,
    plot_result=True,
)

In [ ]:
# END